## **О проекте: Анализ выборов приздента России 2024.**

Проект посвящён анализу факторов, повлиявших на результатыт кандидатов на выборах на конкретных учасках или в регионах, а следовательно и на результат выборов в целом. А также представление удобной и красивой визулизации результатов выборов по регионам.

#   0. Скачивание и установка необходимых библиотек.

In [96]:
!pip install streamlit_folium


In [97]:
import pandas as pd
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap
import seaborn as sns
import requests
from streamlit_folium import st_folium
import streamlit as st
import sqlite3
from bs4 import BeautifulSoup
import numpy as np
import ipywidgets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


# **1. Загрузка данных.**

## Результаты выборов по ДЭГ, территориальным избирательным комиссиям и участковым избирательным комиссиям.

In [100]:
elections_tik = pd.read_csv('/content/drive/MyDrive/Project/results-tik-20240325T0416UTC.tsv',sep='\t')
elections_uik = pd.read_csv('/content/drive/MyDrive/Project/results-uik-20240325T0139UTC.tsv',sep='\t')
elections_deg = pd.read_csv('/content/drive/MyDrive/Project/results-deg-20240325T0428UTC.tsv',sep='\t')



## Парсинг данных о ВРП на душу населения россйиских регионов с сайта Росстата.
 Загрузка страницы сайта

In [101]:
page_url = "https://rosstat.gov.ru/storage/mediabank/Reg_Rus_Pokaz_2023.htm"

# Define headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive"
}

# Fetch the HTML content of the page
response = requests.get(page_url, headers=headers)
response.raise_for_status()

ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

Создаём дерево с помощью BeatifulSoup

In [ ]:
content = response.content
tree = BeautifulSoup(content, 'html.parser')



Выделаем кусочки где, лежат ссылки

In [ ]:
links = tree.find_all('a')

Находим как называется нужная нам ссылка

In [ ]:
links[18].text.strip()

Достаём ссылку

In [ ]:
for link in links:
    if link.text.strip() == 'Валовой\r\n    региональный продукт':
        GRDP_link = link['href']
        print("URL:", link['href'])
        break

Скачиваем файл

In [ ]:
GRDP = requests.get(GRDP_link, headers = headers)
file_path = "RR_pokaz_09_2023.xlsx"
with open(file_path, "wb") as file:
    file.write(GRDP.content)
sheet_name = "9.2"



In [ ]:
GRDP = pd.read_excel(file_path, sheet_name=sheet_name)

## Загрузка карт России и новых регионов.

In [ ]:
gdf = gpd.read_file("/content/drive/MyDrive/Project/russia_regions-2.geojson")


In [ ]:
LPR = gpd.read_file("/content/drive/MyDrive/Project/UA_09_Luhanska.geojson")
LPR.plot(figsize = (3,3))
plt.show()
DPR = gpd.read_file("/content/drive/MyDrive/Project/UA_14_Donetska.geojson")
DPR.plot(figsize = (3,3))
plt.show()
ZAP = gpd.read_file("/content/drive/MyDrive/Project/UA_23_Zaporizka.geojson")
ZAP.plot(figsize = (3,3))
plt.show()
KHE = gpd.read_file("/content/drive/MyDrive/Project/UA_65_Khersonska.geojson")
KHE.plot(figsize = (3,3))
plt.show()


# 2. Подготовка данных

Делаем регион индексом

In [ ]:
elections_deg.set_index('region', inplace=True, drop = True)
elections_tik.set_index('region', inplace=True, drop = True)
elections_uik.set_index('region', inplace=True, drop = True)

В данных по УИКам отсутствуют данные по новым регионам

In [ ]:
elections_uik[elections_uik['Число действительных избирательных бюллетеней'].isnull()]

Избавляемся от пустых строк.

In [ ]:
elections_uik = elections_uik[elections_uik['Число действительных избирательных бюллетеней'].isnull() == 0]

Посчитаем явку на каждом из участков.

In [ ]:
elections_uik['Число выданных избирательных бюллетеней'] = elections_uik['Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно'].add(elections_uik['Число избирательных бюллетеней, выданных в помещении для голосования в день голосования'])
elections_uik['Число выданных избирательных бюллетеней'] = elections_uik['Число выданных избирательных бюллетеней'].add(elections_uik['Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования'])
elections_uik['явка (%)'] = elections_uik['Число выданных избирательных бюллетеней']/elections_uik['Число избирателей, включенных в список избирателей']*100

In [ ]:
elections_uik

## Считаем результаты по регионам из данных по ТИКам(чтобы учесть данные по новым регионам).

In [ ]:
regions = elections_tik.groupby('region')
DAV = regions.sum()['Даванков Владислав Андреевич'].add(elections_deg['Даванков Владислав Андреевич'],fill_value = 0 ) / ((regions.sum()['Число действительных избирательных бюллетеней'].add(elections_deg['Число действительных избирательных бюллетеней'], fill_value = 0 ) ).add(elections_deg['Число недействительных избирательных бюллетеней'], fill_value = 0 )).add(regions.sum()['Число недействительных избирательных бюллетеней']) * 100
PUT = regions.sum()['Путин Владимир Владимирович'].add(elections_deg['Путин Владимир Владимирович'], fill_value = 0 ) / ((regions.sum()['Число действительных избирательных бюллетеней'].add(elections_deg['Число действительных избирательных бюллетеней'], fill_value = 0 ) ).add(elections_deg['Число недействительных избирательных бюллетеней'], fill_value = 0 )).add(regions.sum()['Число недействительных избирательных бюллетеней']) * 100
SLUT = regions.sum()['Слуцкий Леонид Эдуардович'].add(elections_deg['Слуцкий Леонид Эдуардович'], fill_value = 0 ) / ((regions.sum()['Число действительных избирательных бюллетеней'].add(elections_deg['Число действительных избирательных бюллетеней'], fill_value = 0 ) ).add(elections_deg['Число недействительных избирательных бюллетеней'], fill_value = 0 )).add(regions.sum()['Число недействительных избирательных бюллетеней']) * 100
KCH = regions.sum()['Харитонов Николай Михайлович'].add(elections_deg['Харитонов Николай Михайлович (%)'], fill_value = 0) / ((regions.sum()['Число действительных избирательных бюллетеней'].add(elections_deg['Число действительных избирательных бюллетеней'], fill_value = 0 ) ).add(elections_deg['Число недействительных избирательных бюллетеней'], fill_value = 0 )).add(regions.sum()['Число недействительных избирательных бюллетеней']) * 100


In [ ]:
DAV = pd.DataFrame(DAV)
DAV = DAV.rename( columns = {0:'Даванков Владислав Андреевич (%)'})
PUT = pd.DataFrame(PUT)
PUT = PUT.rename( columns = {0:'Путин Владимир Владимирович (%)'})
SLUT = pd.DataFrame(SLUT)
SLUT = SLUT.rename( columns = {0:'Слуцкий Леонид Эдуардович (%)'})
KCH = pd.DataFrame(KCH)
KCH = KCH.rename( columns = {0:'Харитонов Николай Михайлович (%)'})
elections_reg = pd.merge(DAV , PUT, left_index=True, right_index=True)
elections_reg = pd.merge( elections_reg , SLUT, left_index=True, right_index=True )
elections_reg = pd.merge( elections_reg , KCH, left_index=True, right_index=True )



## Добавялем донные о ВРП на душу населения к данным по УИКам

Структура загруженного документа

In [ ]:
GRDP

Убираем ненужные колонки

In [ ]:
GRDP = GRDP.drop( columns = ['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5'])

Переименовываем колонки, для удобства работы.

In [ ]:
GRDP.rename(columns={'9.2. ВАЛОВОЙ РЕГИОНАЛЬНЫЙ ПРОДУКТ на душу населения1) \n(рублей)': 'region', 'Unnamed: 6' : 'GRDP'}, inplace=True)

Убираем неинформативные строки.

In [ ]:
GRDP.drop(index=[0, 99], inplace=True)

Приводим в соответвие названия регионов в разных данных.

In [ ]:
GRDP.loc[GRDP['region'] == 'Республика Адыгея', 'region'] = 'Республика Адыгея (Адыгея)'
GRDP.loc[GRDP['region'] == 'Республика Татарстан', 'region'] = 'Республика Татарстан (Татарстан)'
GRDP.loc[GRDP['region'] == 'Чувашская Республика', 'region'] = 'Чувашская Республика - Чувашия'
GRDP.loc[GRDP['region'] == 'Республика Северная Осетия – Алания', 'region'] = 'Республика Северная Осетия - Алания'
GRDP.loc[GRDP['region'] == 'Ханты-Мансийский автономный  округ – Югра', 'region'] = 'Ханты-Мансийский автономный округ - Югра'
GRDP.loc[GRDP['region'] == 'г. Москва', 'region'] = 'город Москва'
GRDP.loc[GRDP['region'] == 'г. Санкт-Петербург', 'region'] = 'город Санкт-Петербург'
GRDP.loc[GRDP['region'] == 'г. Севастополь', 'region'] = 'город Севастополь'
GRDP.loc[GRDP['region'] == 'Кемеровская область', 'region'] = 'Кемеровская область - Кузбасс'

Новая структура датафрейма

In [ ]:
GRDP

Добавляем данные о ВРП в данные по УИКам

In [ ]:
elections_uik = pd.merge(elections_uik, GRDP, on='region', how='left')

Проверяем, смогли ли мы исправить все регионы (новых регионов в данных по ВРП нету).

In [ ]:
GRDP.set_index('region', inplace=True, drop = True)

In [ ]:
unique_indices = elections_reg.index.difference(GRDP.index)
unique_indices

## Создание карты с новыми регионами

Структура данных с новыми регионами

In [ ]:
LPR

Приводим в соответствие названия новых регионов с данными о выборах

In [ ]:
#Fixing new regions
LPR['name'][0] = 'Луганская Народная Республика'
LPR = LPR.rename( columns = {'name': 'region'})
LPR.set_index('region', inplace = True, drop = True)

DPR['name'][0] = 'Донецкая Народная Республика'
DPR = DPR.rename( columns = {'name': 'region'})
DPR.set_index('region', inplace = True, drop = True)

ZAP['name'][0] = 'Запорожская область'
ZAP = ZAP.rename( columns = {'name': 'region'})
ZAP.set_index('region', inplace = True, drop = True)

KHE['name'][0] = 'Херсонская область'
KHE = KHE.rename( columns = {'name': 'region'})
KHE.set_index('region', inplace = True, drop = True)

Приводим в соответствие названия старых регионов на карте и в данных о выборах

In [ ]:
# Fixing old regions
gdf.loc[gdf['region'] == 'Республика Адыгея', 'region'] = 'Республика Адыгея (Адыгея)'
gdf.loc[gdf['region'] == 'Республика Татарстан', 'region'] = 'Республика Татарстан (Татарстан)'
gdf.loc[gdf['region'] == 'Чувашская Республика', 'region'] = 'Чувашская Республика - Чувашия'
gdf.loc[gdf['region'] == 'Республика Северная Осетия — Алания', 'region'] = 'Республика Северная Осетия - Алания'
gdf.loc[gdf['region'] == 'Ханты-Мансийский автономный округ — Югра', 'region'] = 'Ханты-Мансийский автономный округ - Югра'
gdf.loc[gdf['region'] == 'Москва', 'region'] = 'город Москва'
gdf.loc[gdf['region'] == 'Санкт-Петербург', 'region'] = 'город Санкт-Петербург'
gdf.loc[gdf['region'] == 'Севастополь', 'region'] = 'город Севастополь'
gdf.loc[gdf['region'] == 'Кемеровская область', 'region'] = 'Кемеровская область - Кузбасс'




Задаём регион как индекс

In [ ]:
gdf.set_index('region', inplace=True, drop = True)

Объединяем всё файлы в однку карту.

In [ ]:
SVO = pd.concat([LPR,DPR])
SVO = pd.concat([SVO,KHE])
SVO = pd.concat([SVO,ZAP])
SVO = pd.concat([SVO,gdf])

# 3. Проверка Гипотез.

## Гипотеза №1.
Протестное Голосование.

На протестных участках людям было принципиально лишь проголосовать за кого-либо, кроме президента.
Следовательно, на участках, на которых, люди больше голосовали за одного из кандидатов альтернативных президенту они больше голосовали и за других.

Считаем корреляцию, рисуем скаттер плот и регрессию.

In [ ]:
 def trend ( candidat1 , candidat2, type ):
  if type == 'uik':
    plt.scatter( elections_uik[f'{candidat1} (%)'] , elections_uik[f'{candidat2} (%)'] )
    sns.regplot(x = f'{candidat1} (%)' , y = f'{candidat2} (%)' , data = elections_uik, scatter = False, color = 'red')
    plt.xlim(0 , 100)
    plt.ylim(0 , 100)
    plt.show()
    correl = elections_tik[f'{candidat1} (%)'].corr(elections_tik[f'{candidat2} (%)'] )
    print(correl)
  if type == 'tik':
    plt.scatter( elections_tik[f'{candidat1} (%)'] , elections_tik[f'{candidat2} (%)'] )
    sns.regplot(x = f'{candidat1} (%)' , y = f'{candidat2} (%)' , data = elections_tik, scatter = False, color = 'red')
    plt.xlim(0 , 100)
    plt.ylim(0 , 100)
    plt.show()
    correl = elections_tik[f'{candidat1} (%)'].corr(elections_tik[f'{candidat2} (%)'] )
    print(correl)
  if type == 'reg':
    plt.scatter( elections_reg[f'{candidat1} (%)'] , elections_reg[f'{candidat2} (%)'] )
    sns.regplot(x = f'{candidat1} (%)' , y = f'{candidat2} (%)' , data = elections_reg, scatter = False, color = 'red')
    plt.xlim(0 , 100)
    plt.ylim(0 , 100)
    plt.show()
    correl = elections_reg[f'{candidat1} (%)'].corr(elections_reg[f'{candidat2} (%)'] )
    print(correl)





Кнопки, выбирающие двух кандидатов и территриальную еденицу. ТИК/регион или УИК

In [ ]:

candidat_selector1 = ipywidgets.Dropdown(
    options=('Путин Владимир Владимирович', 'Даванков Владислав Андреевич', 'Слуцкий Леонид Эдуардович', 'Харитонов Николай Михайлович'),
    index=0,
    value='Путин Владимир Владимирович',
    layout={'width': '700px'},
    continuous_update = False
)
candidat_selector2 = ipywidgets.Dropdown(
    options=('Путин Владимир Владимирович', 'Даванков Владислав Андреевич', 'Слуцкий Леонид Эдуардович', 'Харитонов Николай Михайлович'),
    index=0,
    value='Путин Владимир Владимирович',
    layout={'width': '700px'},
    continuous_update = False
)
candidat_selector3 = ipywidgets.Dropdown(
    options=('tik', 'uik', 'reg'),
    index=0,
    value='tik',
    layout={'width': '700px'},
    continuous_update = False
)
ipywidgets.interact(
    trend, candidat1 = candidat_selector1, candidat2 = candidat_selector2, type = candidat_selector3
)

Как мы видим, корреляция между результатами всех оппозиционных кандиадатов положительна. Корреляция же между результатми В. В. Путина и резльтатми оппозиционных кандидатов отрицательна.
Можно считать значение противоположное корреляцит с результатами президента, как коэффициэнт оппозиционности кандидата в глазах общества.
Выходит коэффициэнт оппозиционности Даванкова: 0.95;
Харитонова: 0.22; Слуцкого: 0.24(по УИКам).

## Гипотеза № 2
Пила Памчурова?

На российских выборах в относительно недалеком прошлом имел место феномен "Пилы Чурова". На большом числе участков по странному совпадению результаты были круглые. В данной части я проверю имеет ли место подобный феномен на выборах 2024 года.

In [ ]:
def Churov (candidat1):
  plt.figure(figsize=(15, 6))
  plt.hist(elections_uik[f'{candidat1} (%)'], bins=10000, range=(0, 100), edgecolor='black')
  plt.title('Частота появления каждого значения от 0 до 100')
  plt.xlabel('Значения')
  plt.ylabel('Частота')
  plt.grid(True)
  plt.show()


In [ ]:
def Suspicious_values(candidat):
  value_counts = elections_uik[f'{candidat} (%)'].value_counts()
  frequent_values = value_counts[value_counts > 300].index
  filtered_df = elections_uik[elections_uik[f'{candidat} (%)'].isin(frequent_values)]
  filtered_df = filtered_df[filtered_df[f'{candidat} (%)']!=0]

  return filtered_df

In [ ]:
Churov('Путин Владимир Владимирович')

In [ ]:
suspicious_selector = ipywidgets.Dropdown(
    options=('Путин Владимир Владимирович', 'Даванков Владислав Андреевич', 'Слуцкий Леонид Эдуардович', 'Харитонов Николай Михайлович'),
    index=0,
    value='Путин Владимир Владимирович',
    layout={'width': '700px'},
    continuous_update = False
)
ipywidgets.interact(
    Suspicious_values, candidat = suspicious_selector
)

Как мы видим, хотя некоторые пики и пристутсвуют на графике, сложно говорить о наличии так называемой Пилы.
Пики обладают небольшой частносьтью. А круглые числы зачастую объясняются небольшим числом избирателей на участке.

## Гипотеза № 3
Машинковое обучение

Результат оппозиционных кандидатов больше в более населённых и богатых территориях.

Я попытаюсь выяснить можно ли по ВП региона явке и числу зарегестрированных избирателей на участке предсказать результат каждого кандидата

Функция, тренирующая модель. Также она выдаёт её характеристки.

In [ ]:
def train (candidat):
  target_column = f'{candidat} (%)'

  # Select the necessary columns for prediction
  columns_needed = ['Число избирателей, включенных в список избирателей', 'явка (%)', 'GRDP', target_column]
  data = elections_uik[columns_needed]

    # Drop rows with any missing values
  data.dropna(inplace=True)

  # Separate the features and the target variable
  x = data.drop(columns=[target_column])
  y = data[target_column]

  # Split the data into training and testing sets
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

  # Train a predictive model
  model = LinearRegression()
  model.fit(x_train, y_train)

  # Evaluate the model
  y_pred = model.predict(x_test)
  mse = mean_squared_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)
  print(f'Mean Squared Error: {mse}')
  print(f'R^2 Score: {r2}')
  print(f'Model Coefficients: {model.coef_}')
  return model

Функция, выдающая предсказание, на оснвое модели

In [ ]:
def prediction (population, come, GRDP, candidat):
  new_data = {
    'Число избирателей, включенных в список избирателей': [population],
    'явка (%)': [come],
    'GRDP': [GRDP]
    }
  new_df = pd.DataFrame(new_data)
  model = train(candidat)
  predictions = model.predict(new_df)
  return predictions



In [ ]:
prediction(5000, 80, 80000,'Путин Владимир Владимирович')

Считаем дисперсию для оценки эффектвиности оценки. Она больше, чем MSE, а знчит оценка достатчоно эффективна.

In [ ]:
np.var(elections_uik['Путин Владимир Владимирович (%)'])

Можно заметить, что предсказанный результат президента растёт от явки и снижается от ВРП и численности избирателей. Результат оппозиционных результатов, наоборот падает от явки и растёт от ВРП и падает от явки. =>
Гипотеза подтверждена

## Гипотеза №4

Результаты кандидатов значительно отличаются от региона к региону.

In [ ]:
elections_reg['region'] = elections_reg.index

Функция рисует карту.

In [ ]:
def elections_map (candidat):
  RU_Coordinates = (58, 105)
  Rus = folium.Map(location=RU_Coordinates, zoom_start=3)
  folium.Choropleth(
    geo_data = SVO,
    data = elections_reg ,
    columns=["region", f"{candidat} (%)"],
    key_on = "feature.id",
    fill_color = "YlGnBu",
    highlight = True,
    legend_name = f'{candidat} %'
  ).add_to(Rus)
  return Rus

Кнопочка для выбора кандидата, по которому строится карта

In [ ]:
candidat_selector = ipywidgets.Dropdown(
    options=('Путин Владимир Владимирович', 'Даванков Владислав Андреевич', 'Слуцкий Леонид Эдуардович', 'Харитонов Николай Михайлович'),
    index=0,
    value='Путин Владимир Владимирович',
    layout={'width': '700px'},
    continuous_update = False
)
ipywidgets.interact(
    elections_map, candidat = candidat_selector
)

Как мы видим на карте, результаты кандидатов значтельно отличаются от резльтатов в других регионах. Например по каким-то причинам в Кузбассе и Туве процент В.В Путина значительно выше, чем в большинстве регионов.